# Missing packages

In [ ]:
!pip install transformers --quiet

     |████████████████████████████████| 675kB 3.5MB/s 
     |████████████████████████████████| 3.8MB 8.6MB/s 
     |████████████████████████████████| 1.1MB 39.8MB/s 
     |████████████████████████████████| 890kB 39.9MB/s 


# Imports

In [ ]:
import os, sys, gc, ast
import string
import pandas as pd
import numpy as np

In [ ]:
from transformers import RobertaTokenizer, BertTokenizer

# Data

In [ ]:
path = '../data/'

In [ ]:
train = pd.read_csv(path+'Train.csv')
test = pd.read_csv(path+'Test.csv')

In [ ]:
df = pd.concat([train, test])

In [ ]:
train.head()

,ID,text,label
0,SUAVK39Z,I feel that it was better I dieAm happy,Depression
1,9JDAGUV3,Why do I get hallucinations?,Drugs
2,419WR1LQ,I am stresseed due to lack of financial suppor...,Depression
3,6UY7DX6Q,Why is life important?,Suicide
4,FYC0FTFB,How could I be helped to go through the depres...,Depression


In [ ]:
test.head()

,ID,text
0,02V56KMO,How to overcome bad feelings and emotions
1,03BMGTOK,I feel like giving up in life
2,03LZVFM6,I was so depressed feel like got no strength t...
3,0EPULUM5,I feel so low especially since I had no one to...
4,0GM4C5GD,can i be successful when I am a drug addict?


In [ ]:
df.head()

,ID,text,label
0,SUAVK39Z,I feel that it was better I dieAm happy,Depression
1,9JDAGUV3,Why do I get hallucinations?,Drugs
2,419WR1LQ,I am stresseed due to lack of financial suppor...,Depression
3,6UY7DX6Q,Why is life important?,Suicide
4,FYC0FTFB,How could I be helped to go through the depres...,Depression


#Tokenizer

In [ ]:
!mkdir tokenizer

In [ ]:
tok = RobertaTokenizer.from_pretrained('roberta-base')

In [ ]:
tok.save_pretrained('tokenizer/')

('tokenizer/vocab.json',
 'tokenizer/merges.txt',
 'tokenizer/special_tokens_map.json',
 'tokenizer/added_tokens.json')

In [ ]:
with open('tokenizer/vocab.json', 'r') as f:
  vocab = ast.literal_eval(f.read())
  f.close()

In [ ]:
len(vocab)

50265

In [ ]:
# vocab.keys()

#Preprocessing

In [ ]:
def step_0(x):
  x = str(x)
  x = " " + x
  x = x.lower()
  return x

In [ ]:
df['text'] = df.text.apply(step_0)

In [ ]:
punct = '!,.:;?@[]{|}~-…()'

def step_1(x):
  for p in punct:
    x = x.replace(p, " "+p+" ")
  return x

In [ ]:
df['text'] = df.text.apply(step_1)

In [ ]:
def step_2(x):
  x = x.replace("i'm ", "i am")
  x = x.replace("\'s ", " is", )
  x = x.replace("\'re ", " are", )
  x = x.replace("\'ve ", " have")
  x = x.replace("can't ", "cannot")
  x = x.replace("n't ", " not")
  x = x.replace("\'d", " would")
  x = x.replace("\'ll", " will")
  return x

In [ ]:
df['text'] = df.text.apply(step_2)

# Vocab

In [ ]:
df_vocab_ = sorted(list( set( ' '.join(df.text.values.tolist()).split() ) ))

In [ ]:
len(df_vocab_)

1079

In [ ]:
df_vocab_[:12]

["'", '(', ')', ',', '-', '.', '2', '?', '[', 'a', 'abandoned', 'ability']

In [ ]:
df_vocab = df_vocab_[9:]

In [ ]:
df_vocab[:10]

['a',
 'abandoned',
 'ability',
 'able',
 'abort',
 'about',
 'absent',
 'abstain',
 'abuse',
 'abusing']

In [ ]:
df_vocab[-5:]

['you', 'young', 'your', 'youths', '…']

In [ ]:
encs = [ tok.encode(x) for x in df_vocab]

In [ ]:
sizs =  [len(x) for x in encs]

In [ ]:
crits = np.where(np.array(sizs) != 3, 1, 0)

In [ ]:
oot = np.unique(np.where(crits==1, df_vocab, ''))

In [ ]:
oot

array(['', 'abstain', 'activites', 'addidcted', 'ahd', 'alccohol',
       'ambetter', 'amfeeling', 'amnot', 'amokay', 'amquite', 'amstill',
       'amtaking', 'amunworthy', 'annd', 'ather', 'avoiod', 'benefitial',
       'benefitto', 'bette', 'bhang', 'birthnow', 'blacking', 'burnout',
       'calmness', 'cannothandle', 'cannotstop', 'childlish', 'classwork',
       'commited', 'commiting', 'confusednow', 'conqured', 'consented',
       'contentment', 'contol', 'cornerfeeling', 'counsellor', 'defering',
       'deferrin', 'demotivated', 'depresses', 'depressioni', 'desparate',
       'despising', 'desserted', 'deteroriating', 'didn’t', 'dieam',
       'dillusioned', 'diserted', 'dissapointed', 'dissappointed',
       'dizzines', 'don’t', 'doto', 'downhearted', 'downplayed',
       'downrecovering', 'drinnking', 'drugsnow', 'drunkard',
       'everythingi', 'existd', 'faints', 'fantasizing', 'felling', 'fgm',
       'finacial', 'foind', 'fretting', 'frorge', 'frustratedi', 'gf',
       

In [ ]:
fix = {
'dieam':'die am',
'stresseed':'stressed',
'drugsnow':'drugs now',
'frorge':'forge',
'mediataton':'meditation',
'hatrednow':'hatred now',
'cornerfeeling':'corner feeling',
'avoiod':'avoid',
'negletion':'neglection',
'lonelynow':'lonely now',
'isolatednow':'isolated now',
'confusednow':'confused now',
'hopelessfor':'hopeless for',
'havei':'have i',
'everythingi':'everything i',
'thingof':'thing of',
'liferight':'life right',
'deteroriating':'deteriorating',
'stresssed':'stressed',
'lonelyi':'lonely i',
'childlish':'childish',
'depressioni':'depression i',
'addidcted':'addicted',
'alccohol':'alcohol',
'drinnking':'drinking',
'schoolfees':'school fees',
'diserted':'deserted',
'lonelycurrently':'lonely currently',
'existd':'existed',
'schoolfee':'school fee',
'negativecurrently':'negative currently',
'messnow':'mess now',
'frustratedi':'frustrated i',
'benefitto':'benefit to',
'weathernow':'weather now',
'incidencesof':'incidences of',
'worldm':'world',
'unworthyness':'unworthiness',
'oldself':'old self',
'undestands':'understands',
'issolated':'isolated',
'dillusioned':'delusional',
'hopelessnesss':'hopelessness',
'includingmy':'including my',
'motivationsuicidal':'motivation suicidal',
'birthnow':'birth now',
'helplessstill':'helpless still',
'lowt':'low',
'helplessi':'helpless i',
'deferrin':'deferring',
'deferringgg':'deferring',
'deferringg': 'deferring',
'defering':'deferring',
'downrecovering':'down recovering',
'nightsi':'nights i',
'worthlesness':'worthlessness',
'ponographic': 'pornographic',
'sucidal':'suicidal',
'dizzines':'dizziness',
'insomia':'insomnia',
'conqured':'conquered',
'worhtless':'worthless',
'selfworth':'self worth',
'moderatly':'moderately',
'desserted':'deserted',
'patners':'partners',
'psycologically': 'psychologically',
'sove':'solve',
'foind':'found',
'lowi': 'low i',
'whren': 'when',
}

In [ ]:
unks = []
knowns = []
for o in oot:
  try:
    a = fix[o]
    knowns.append(o)
  except KeyError:
    unks.append(o)

In [ ]:
np.array(unks)

array(['', 'abstain', 'activites', 'ahd', 'ambetter', 'amfeeling',
       'amnot', 'amokay', 'amquite', 'amstill', 'amtaking', 'amunworthy',
       'annd', 'ather', 'benefitial', 'bette', 'bhang', 'blacking',
       'burnout', 'calmness', 'cannothandle', 'cannotstop', 'classwork',
       'commited', 'commiting', 'consented', 'contentment', 'contol',
       'counsellor', 'demotivated', 'depresses', 'desparate', 'despising',
       'didn’t', 'dissapointed', 'dissappointed', 'don’t', 'doto',
       'downhearted', 'downplayed', 'drunkard', 'faints', 'fantasizing',
       'felling', 'fgm', 'finacial', 'fretting', 'gf', 'havelost',
       'he/she', 'heartbreak', 'heartbreaks', 'heartbroken',
       'helplessness', 'highschool', 'hustle', 'hw', 'impregnated',
       'inadequacy', 'indescribable', 'indulgence', 'inorder', 'instable',
       'isdepression', 'isnext', 'ith', 'loneliest', 'looses', 'lossing',
       'lowkey', 'migraines', 'misbehave', 'miscarriages', 'motherhood',
       'motivat

In [ ]:
np.array(knowns)

array(['addidcted', 'alccohol', 'avoiod', 'benefitto', 'birthnow',
       'childlish', 'confusednow', 'conqured', 'cornerfeeling',
       'defering', 'deferrin', 'depressioni', 'desserted',
       'deteroriating', 'dieam', 'dillusioned', 'diserted', 'dizzines',
       'downrecovering', 'drinnking', 'drugsnow', 'everythingi', 'existd',
       'foind', 'frorge', 'frustratedi', 'hatrednow', 'havei',
       'helplessi', 'helplessstill', 'hopelessfor', 'hopelessnesss',
       'incidencesof', 'includingmy', 'insomia', 'isolatednow',
       'issolated', 'liferight', 'lonelycurrently', 'lonelyi',
       'lonelynow', 'lowi', 'lowt', 'mediataton', 'messnow', 'moderatly',
       'motivationsuicidal', 'negativecurrently', 'negletion', 'nightsi',
       'oldself', 'patners', 'ponographic', 'psycologically', 'schoolfee',
       'schoolfees', 'selfworth', 'sove', 'stresseed', 'stresssed',
       'sucidal', 'thingof', 'undestands', 'unworthyness', 'weathernow',
       'whren', 'worhtless', 'worldm', '

In [ ]:
def viz(ws):
  for w in ws:
    enc = tok.encode(w)
    dec = tok.decode(enc)
    dec_ = [ tok.decode([x]) for x in enc[1:-1]]
    print('[WORD] {} [ENC] {} [DEC] {} [DEC V2] {} '.format(w, enc, dec, dec_))

In [ ]:
# viz(knowns)

In [ ]:
# viz(unks[1:])

In [ ]:
fix2 = {
'ahd':'had',
'ambetter':'am better',
'amfeeling':'am feeling',
'amnot':'am not',
'amokay':'am okay',
'amquite':'am quite',
'amstill':'am still',
'amtaking':'am taking',
'amunworthy':'am unworthy',
'annd':'and',
'ather':'rather',
'benefitial':'beneficial',
'bette ':'better ', 
'cannothandle':'cannot handle',
'cannotstop':'cannot stop',
'commited':'committed',
'commiting':'committing',
'contol':'control',
'counsellor':'counselor',
'demotivated':'demotivated',
'depresses':'depresses',
'desparate':'desperate',
'despising':'despising', 
'dissapointed':'disappointed',
'dissappointed':'disappointed',
'doto':'do to',
'felling':'feeling',
'fgm':'female genital mutilation',
'finacial':'financial',
'gf ':'girlfriend',
'havelost':'have lost',
# 'he/she':'he/she',
'highschool':'highschool',
'hustle':'hustle',
'hw':'how',
'inorder':'in order',
'instable':'instable',
'isdepression':'is depression',
'isnext':'is next',
'ith ':'with',
'lossing':'loosing',
'notbelong':'not belong',
'notfeel':'not feel',
'notover':'not over',
'notsee':'not see',
'nott ':'not',
'notwant':'not want',
'npt':'not',
'reconcilliation':'reconciliation',
'remover':'remover',
'sadi':'sad i',
'someon':'someone',
'sould':'should',
'stagnated':'stagnated',
'stop/':'stop /',
'succed':'succeed',
'wellwishers':'well wishers',
'whn':'when', 
}

# Final cleaning

In [ ]:
def cleaning(x):
  x = step_0(x)
  x = step_1(x)

  for p in fix.keys():
      x = x.replace(p, fix[p])
  for p in fix2.keys():
      x = x.replace(p, fix2[p])

  return " " + x.strip(" ")

In [ ]:
train['text'] = train.text.apply(cleaning)
test['text'] = test.text.apply(cleaning)

In [ ]:
train.head()

,ID,text,label
0,SUAVK39Z,i feel that it was better i die am happy,Depression
1,9JDAGUV3,why do i get hallucinations ?,Drugs
2,419WR1LQ,i am stressed due to lack of financial suppor...,Depression
3,6UY7DX6Q,why is life important ?,Suicide
4,FYC0FTFB,how could i be helped to go through the depre...,Depression


In [ ]:
train = train.drop_duplicates(['text'])

In [ ]:
train.reset_index(drop=True, inplace=True)

# Dummify the label

In [ ]:
labels = ['Depression', 'Alcohol', 'Suicide', 'Drugs']

In [ ]:
for label in labels:
  train[label] = train['label'].apply(lambda x: int(x==label))

In [ ]:
mapper = {'Depression': 0,
          'Alcohol': 1,
          'Suicide': 2,
          'Drugs': 3}
train['label'] = train.label.map(mapper)

In [ ]:
train.head()

,ID,text,label,Depression,Alcohol,Suicide,Drugs
0,SUAVK39Z,i feel that it was better i die am happy,0,1,0,0,0
1,9JDAGUV3,why do i get hallucinations ?,3,0,0,0,1
2,419WR1LQ,i am stressed due to lack of financial suppor...,0,1,0,0,0
3,6UY7DX6Q,why is life important ?,2,0,0,1,0
4,FYC0FTFB,how could i be helped to go through the depre...,0,1,0,0,0


# Save everything

In [ ]:
train.to_csv(path+'final_train.csv', index=False)
test.to_csv(path+'final_test.csv', index=False)